In [ ]:
import cv2
import tkinter as tk
from threading import Thread
from PIL import Image, ImageTk
import time

class MainApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Main Page")
        self.root.geometry("400x300")

        # Load and display the background image
        self.bg_image = Image.open("main.jpg")
        self.bg_image = self.bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)
        self.background_label = tk.Label(root, image=self.bg_photo)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)

        # Store the PhotoImage object as an instance attribute to avoid garbage collection
        self.background_label.image = self.bg_photo

        # Create and place "About Us" button
        self.about_us_button = tk.Button(root, text="About Us", command=self.show_about_us, bg='yellow', font=18, width=20)
        self.about_us_button.place(relx=0.5, rely=0.70, anchor=tk.CENTER)

        # Create and place "Try" button
        self.try_button = tk.Button(root, text="Try", command=self.open_cheating_monitor, bg='yellow', font=18, width=20)
        self.try_button.place(relx=0.5, rely=0.85, anchor=tk.CENTER)

    def show_about_us(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        self.app = AboutUsPage(self.root)

    def open_cheating_monitor(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        self.app = CheatingMonitorApp(self.root)

class AboutUsPage:
    def __init__(self, root):
        self.root = root
        self.root.title("About Us")

        # Load and display the background image
        self.bg_image = Image.open("About_us.jpg")  # Use a suitable image for the About Us page
        self.bg_image = self.bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)
        self.background_label = tk.Label(root, image=self.bg_photo)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)

        # Store the PhotoImage object as an instance attribute to avoid garbage collection
        self.background_label.image = self.bg_photo

        # Create and place the Back button
        self.back_button = tk.Button(root, text="Back", command=self.back_to_main, bg='light blue', font=18, width=20)
        self.back_button.place(relx=0.1, rely=0.95, anchor=tk.SW)

    def back_to_main(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        MainApp(self.root)

class CheatingMonitorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Cheating Monitoring and Eye Tracking")

        self.is_running = False
        self.cheating_start_time = None
        self.cheating_detected = False

        # Load and display the background image
        self.bg_image = Image.open("Background.jpg")
        self.bg_image = self.bg_image.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)
        self.background_label = tk.Label(root, image=self.bg_photo)
        self.background_label.place(x=0, y=0, relwidth=1, relheight=1)

        # Store the PhotoImage object as an instance attribute to avoid garbage collection
        self.background_label.image = self.bg_photo

        # Create and place start button
        self.start_button = tk.Button(root, text="Start Monitoring", command=self.start_monitoring, bg='yellow', font=18, width=20)
        self.start_button.place(relx=0.5, rely=0.55, anchor=tk.CENTER)

        # Create and place stop button
        self.stop_button = tk.Button(root, text="Stop Monitoring", command=self.stop_monitoring, state=tk.DISABLED, bg='yellow', font=18, width=20)
        self.stop_button.place(relx=0.5, rely=0.70, anchor=tk.CENTER)

        # Create and place status label
        self.status_label = tk.Label(root, text="Status: Not Running", font=18, bg='yellow', width=20)
        self.status_label.place(relx=0.5, rely=0.85, anchor=tk.CENTER)

        # Create and place back button
        self.back_button = tk.Button(root, text="Back", command=self.back_to_main, bg='white', font=18, width=20)
        self.back_button.place(relx=0.1, rely=0.95, anchor=tk.SW)

    def start_monitoring(self):
        self.is_running = True
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.status_label.config(text="Status: Running")
        self.monitor_thread = Thread(target=self.monitor_and_track_cheating)
        self.monitor_thread.start()

    def stop_monitoring(self):
        self.is_running = False
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.status_label.config(text="Status: Not Running")

    def monitor_and_track_cheating(self):
        # Open the default webcam
        cap = cv2.VideoCapture(0)

        # Load the face and eye cascade classifiers
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

        try:
            while self.is_running:
                # Capture frame-by-frame
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert frame to grayscale
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detect faces
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

                # Initialize variables for cheating detection
                cheating_gaze = 0

                # Loop through detected faces
                for (x, y, w, h) in faces:
                    # Region of interest for face in grayscale and color
                    roi_gray = gray[y:y+h, x:x+w]
                    roi_color = frame[y:y+h, x:x+w]

                    # Detect eyes within the face region
                    eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5, minSize=(15, 15))
                    eye_centers = []
                    for (ex, ey, ew, eh) in eyes:
                        # Calculate center of the eyes
                        eye_center = (x + ex + ew // 2, y + ey + eh // 2)
                        eye_centers.append(eye_center)

                    # Check for cheating eye gazing
                    for eye_center in eye_centers:
                        if eye_center[0] < x + w * 0.5 or eye_center[0] > x + w * 0.9:  # Gazing away from center
                            cheating_gaze += 1

                    # Check if cheating detected
                    if cheating_gaze > 0:
                        if self.cheating_start_time is None:
                            self.cheating_start_time = time.time()
                        elif time.time() - self.cheating_start_time > 10:  # More than 10 seconds
                            self.cheating_detected = True
                    else:
                        self.cheating_start_time = None
                        self.cheating_detected = False

                # Calculate percentage of cheating gaze
                total_faces = len(faces)
                if total_faces > 0:
                    gaze_percentage = (cheating_gaze / total_faces) * 100
                else:
                    gaze_percentage = 0

                # Check if all conditions are met and change frame color accordingly
                if gaze_percentage > 50:
                    frame_color = (0, 0, 255)  # Red (Cheating)
                    status_text = "Cheating Detected"
                else:
                    frame_color = (0, 255, 0)  # Green (Not Cheating)
                    status_text = "Not Cheating"

                # Draw percentage text on the frame
                cv2.putText(frame, f"Gaze: {gaze_percentage:.2f}%", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, status_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

                # Draw rectangle around faces with frame color
                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), frame_color, 2)

                # Display the resulting frame
                cv2.imshow('Cheating Monitoring and Eye Tracking', frame)

                # Check for 'q' key press or window close event
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    self.stop_monitoring()
                    break
                if cv2.getWindowProperty('Cheating Monitoring and Eye Tracking', cv2.WND_PROP_VISIBLE) < 1:
                    self.stop_monitoring()
                    break
        finally:
            # Ensure the capture is released and windows are destroyed
            cap.release()
            cv2.destroyAllWindows()

    def back_to_main(self):
        for widget in self.root.winfo_children():
            widget.destroy()
        MainApp(self.root)

if __name__ == "__main__":
    root = tk.Tk()
    main_app = MainApp(root)
    root.mainloop()
